In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from config import *
from load_data import fetch_data, fetch_trajectory, fetch_player_box
from Extractor.RoundMatcher import find_match_round_dtw, find_match_round_dtw_kmp
from Extractor.utils import compute_stat, interplate_and_align

# DataLoader

In [2]:
import os

file_list = os.listdir(DATA_DIR)
file_list = list(filter(lambda x: "AD" in x, file_list))
file_list

['24082905_AD.csv',
 '24071512_AD.csv',
 '24070902_AD.csv',
 '24090919_AD.csv',
 '24090915_AD.csv',
 '24090917_AD.csv',
 '24090513_AD.csv',
 '24071009_AD.csv',
 '24072825_AD.csv',
 '24090309_AD.csv',
 '24070904_AD.csv',
 '24071617_AD.csv',
 '24082803_AD.csv',
 '24071721_AD.csv',
 '24071619_AD.csv',
 '24071615_AD.csv',
 '24090311_AD.csv',
 '24071011_AD.csv',
 '24070906_AD.csv',
 '24082301_AD.csv',
 'Test_AD.csv',
 '24090918_AD.csv',
 '24090914_AD.csv',
 '24071513_AD.csv',
 '24072926_AD.csv',
 '24082904_AD.csv',
 '24090308_AD.csv',
 '24071008_AD.csv',
 '24090412_AD.csv',
 '24082906_AD.csv',
 '24090916_AD.csv',
 '24070901_AD.csv',
 '24072023_AD.csv',
 '24071616_AD.csv',
 '24083007_AD.csv',
 '24070905_AD.csv',
 '24082302_AD.csv',
 '24070907_AD.csv',
 '24071010_AD.csv',
 '24071618_AD.csv',
 '24090310_AD.csv']

In [3]:
drop_list = ['pingpang.csv', 'tennis.csv', '.DS_Store', 'ControlGroupInfo.xlsx',]

In [4]:
all_data = fetch_data(DATA_DIR, file_list, drop_list)
ball_data = fetch_trajectory(DATA_DIR)
player_box_data = fetch_player_box(os.path.join(DATA_DIR, "PlayerDetectionRes"))

# Feature Extraction

## Saccade Feature

In [5]:
def compute_saccade_path_lite(df:pd.DataFrame):
    _speeds = []
    _angles = []
    indices = list(df.index)
    _dura = (1 * EYE_SAMPLE_TIME ) / 1000.0
    df["Screen.x"] = df["Screen.x"] * VR_SCALE
    df["Screen.y"] = df["Screen.y"] * VR_SCALE

    for _i in range(1, len(indices)):
        row_1 = df.loc[indices[_i - 1], :]
        row_2 = df.loc[indices[_i], :]

        _dist = np.sqrt((row_1["Screen.x"] - row_2["Screen.x"])**2 + (row_1["Screen.y"] - row_2["Screen.y"])**2)
        _angle = np.arctan(_dist / VR_ZDIST) / np.pi * 180

        _angles.append(_angle)
        _speeds.append(np.divide(_angle, _dura))

    return _speeds, _angles

In [6]:
def compute_saccade_path(df:pd.DataFrame, ball_data:pd.DataFrame):
    _speeds = []
    indices = list(df.index)
    _dura = (1 * EYE_SAMPLE_TIME ) / 1000.0
    df["Screen.x"] = df["Screen.x"] * VR_SCALE
    df["Screen.y"] = df["Screen.y"] * VR_SCALE

    ball_data["Ball.x"] = ball_data["Ball.x"] * VR_SCALE
    ball_data["Ball.y"] = ball_data["Ball.y"] * VR_SCALE

    for _i in range(1, len(indices)):
        row_1 = df.loc[indices[_i - 1], :]
        row_2 = df.loc[indices[_i], :]

        _dist = np.sqrt((row_1["Screen.x"] - row_2["Screen.x"])**2 + (row_1["Screen.y"] - row_2["Screen.y"])**2)
        _angle = np.arctan(_dist / VR_ZDIST) / np.pi * 180
        _speeds.append(np.divide(_angle, _dura))

    _eye_dist = np.sqrt((df.iloc[0, :]["Screen.x"] - df.iloc[-1, :]["Screen.x"])**2 + (df.iloc[0, :]["Screen.y"] - df.iloc[-1, :]["Screen.y"])**2)
    _eye_amp = np.arctan(_eye_dist / VR_ZDIST) / np.pi * 180
    _ball_dist = np.sqrt((ball_data.iloc[0, :]["Ball.x"] - ball_data.iloc[-1, :]["Ball.x"])**2 + (ball_data.iloc[0, :]["Ball.y"] - ball_data.iloc[-1, :]["Ball.y"])**2)
    _ball_amp = np.arctan(_ball_dist / VR_ZDIST) / np.pi * 180

    return _speeds, _eye_amp, _eye_amp/_ball_amp

## Trajectory Features

In [7]:

def extract_saccade_features_lite(round_index, data:pd.DataFrame, ball_data_df:pd.DataFrame):
    _fea = {}

    _speeds, _angles = compute_saccade_path_lite(data.copy())
    _fea = {**_fea, **compute_stat("SaccadeSpeed", _speeds)}
    _fea = {**_fea, **compute_stat("SaccadeAngel", _angles)}

    round_start_index = ball_data_df[ball_data_df["round"]==round_index].index[0]
    round_dura = ball_data_df[ball_data_df["round"]==round_index].shape[0]
    eye_start_index = data.index[0]
    _fea["SaccadeDelay"] = (eye_start_index-round_start_index) * EYE_SAMPLE_TIME
    _fea["SaccadeDelayPercent"] = ((eye_start_index-round_start_index) / round_dura) * EYE_SAMPLE_TIME

    return _fea

In [8]:
from Extractor.utils import compute_dtw

def extract_trajectory_lite(round_index, data:pd.DataFrame, ball_data_df):
    _fea = {}
    
    _ball_df = ball_data_df[ball_data_df["round"]==round_index]
    _dtw = compute_dtw(
            line_1=data.loc[:, ["Screen.x", "Screen.y"]],
            line_2=_ball_df.loc[:, ["Ball.x", "Ball.y"]],
            # scale_to_percentage=True,
            # scale_metrics="start"
        )
    
    _fea["TrajectoryDTW"] = _dtw

    return _fea

In [9]:

def add_saccade_features_lite(round_index, data:pd.DataFrame, ball_data_df:pd.DataFrame):
    _fea = {}

    _speeds, _ampli, _unitampli = compute_saccade_path(data.copy(), ball_data_df[ball_data_df["round"]==round_index].copy())
    _fea = {**_fea, **compute_stat("SaccadeSpeed", _speeds)}
    _fea["Amplitude"] = _ampli
    _fea["UnitAmplitude"] = _unitampli

    round_start_index = ball_data_df[ball_data_df["round"]==round_index].index[0]
    round_dura = ball_data_df[ball_data_df["round"]==round_index].shape[0]
    eye_start_index = data.index[0]
    _fea["SaccadeDelay"] = (eye_start_index-round_start_index) * EYE_SAMPLE_TIME
    _fea["SaccadeDelayPercent"] = ((eye_start_index-round_start_index) / round_dura) * EYE_SAMPLE_TIME

    return _fea

In [20]:
from Extractor.utils import compute_dtw

def compute_ball_move(ball_line:pd.DataFrame):
    _indeices = ball_line.index.to_list()
    _dist = []
    for i in range(1, len(_indeices)):
        _dist.append(
                np.sqrt(
                        (ball_line.loc[_indeices[i], "Ball.x"] - ball_line.loc[_indeices[i-1], "Ball.x"])**2 + (ball_line.loc[_indeices[i],"Ball.y"] - ball_line.loc[_indeices[i-1],"Ball.y"])**2
                    )
            )
    return np.sum(_dist)

def compute_two_traj_angle(eye_traj:pd.DataFrame, ball_traj:pd.DataFrame):
    # Line 1 points
    x1, y1 = eye_traj.iloc[0, :]["Screen.x"], eye_traj.iloc[0, :]["Screen.y"]
    x2, y2 = eye_traj.iloc[-1, :]["Screen.x"], eye_traj.iloc[-1, :]["Screen.y"]

    # Line 2 points
    x3, y3 = ball_traj.iloc[0, :]["Ball.x"], ball_traj.iloc[0, :]["Ball.y"]
    x4, y4 = ball_traj.iloc[-1, :]["Ball.x"], ball_traj.iloc[-1, :]["Ball.y"]

    # Create vectors for both lines
    v1 = np.array([x2 - x1, y2 - y1])
    v2 = np.array([x4 - x3, y4 - y3])

    # Calculate the dot product
    dot_product = np.dot(v1, v2)

    # Calculate magnitudes of both vectors
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)

    # Compute the cosine of the angle
    cos_theta = dot_product / (magnitude_v1 * magnitude_v2)

    # Compute the angle in radians
    angle_radians = np.arccos(cos_theta)

    # Convert the angle to degrees
    angle_degrees = np.degrees(angle_radians)

    return angle_degrees

def add_trajectory_lite(round_index, data:pd.DataFrame, ball_data_df):
    _fea = {}
    
    _ball_df = ball_data_df[ball_data_df["round"]==round_index]
    _dtw = compute_dtw(
            line_1=data.loc[:, ["Screen.x", "Screen.y"]],
            line_2=_ball_df.loc[:, ["Ball.x", "Ball.y"]],
        )
    
    _ball_move = compute_ball_move(_ball_df.loc[:, ["Ball.x", "Ball.y"]].copy())

    _fea["TrajDTW"] = _dtw * VR_SCALE
    _fea["TrajDTWPerBallMove"] = (_dtw * VR_SCALE) / (_ball_move * VR_SCALE)
    _fea["DirecAngle"] = compute_two_traj_angle(
            eye_traj=data.loc[:, ["Screen.x", "Screen.y"]],
            ball_traj=_ball_df.loc[:, ["Ball.x", "Ball.y"]].copy()
        )

    return _fea

## Main extrat features function

In [21]:
def extract_features_round(rounds:dict, data_df:pd.DataFrame, ball_data_df:pd.DataFrame):
    data_df["frame"] = data_df.index
    aligned_df = interplate_and_align(data_df, ball_data_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)

    res = {}
    for round_index, round_indices in rounds.items():
        round_res = extract_saccade_features_lite(
            round_index,
            aligned_df.loc[round_indices, ["Screen.x", "Screen.y"]],
            aligned_df.loc[:, ["Ball.x", "Ball.y", "round"]]
            )
        
        traj_res = extract_trajectory_lite(
            round_index,
            aligned_df.loc[round_indices, ["Screen.x", "Screen.y"]],
            aligned_df.loc[:, ["Ball.x", "Ball.y", "round"]]
            )
        res[round_index] = {**round_res, **traj_res}
    return res

In [22]:
def add_saccade_features_round(rounds:dict, data_df:pd.DataFrame, ball_data_df:pd.DataFrame):
    data_df["frame"] = data_df.index
    aligned_df = interplate_and_align(data_df, ball_data_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)

    res = {}
    for round_index, round_indices in rounds.items():
        round_res = add_saccade_features_lite(
            float(round_index),
            aligned_df.loc[round_indices, ["Screen.x", "Screen.y"]],
            aligned_df.loc[:, ["Ball.x", "Ball.y", "round"]]
            )
        
        traj_res = add_trajectory_lite(
            float(round_index),
            aligned_df.loc[round_indices, ["Screen.x", "Screen.y"]],
            aligned_df.loc[:, ["Ball.x", "Ball.y", "round"]]
            )
        res[round_index] = {**round_res, **traj_res}
    return res

In [23]:
from Extractor.utils import max_circle_radius

def threshold_find_match_round_dtw(eye_data:pd.DataFrame, ball_data_df:pd.DataFrame, order, scale_raw_data, mode="fast", dtw_th=1, dist_th=10):
    if mode=="fast":
        rounds, dtw_res = find_match_round_dtw_kmp(eye_data_df=eye_data, ball_data_df=ball_data_df, order=order, scale_raw_data=scale_raw_data)
    elif mode=="greedy":
        rounds, dtw_res = find_match_round_dtw(eye_data_df=eye_data, ball_data_df=ball_data_df, order=order, scale_raw_data=scale_raw_data)

    res = {}
    for _round, _round_index in rounds.items():
        if (max_circle_radius(eye_data.loc[_round_index, :]) >= dist_th) and (dtw_res[_round] <= dtw_th):
            res[_round] = _round_index

    return res, rounds

In [24]:
def judge_inbox(row):
    TOL = 0
    _in_x = (row["Screen.x"] <= row["RightBottom.x"]+TOL) and (row["Screen.x"] >= row["LeftUp.x"]-TOL)
    _in_y = (row["Screen.y"] <= row["LeftUp.y"]+TOL) and (row["Screen.y"] >= row["RightBottom.y"]-TOL)
    return 1 if (_in_x and _in_y) else 0

def compute_inbox_dist(row):
    circle_x = (row["RightBottom.x"] + row["LeftUp.x"]) / 2
    circle_y = (row["LeftUp.y"] + row["RightBottom.y"]) / 2
    return np.sqrt((row["Screen.x"] - circle_x)**2 + (row["Screen.y"] - circle_y)**2) * VR_SCALE 


def extract_features_whole(eye_data:pd.DataFrame, ball_data_df:pd.DataFrame, player_box_data:dict):
    res = {}
    
    p1_bbox_df = pd.DataFrame(player_box_data["Player-1"])
    p2_bbox_df = pd.DataFrame(player_box_data["Player-2"])

    eye_data["frame"] = eye_data.index
    # aligned_df = interplate_and_align(eye_data, ball_data_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)
    aligned_p1_df = interplate_and_align(eye_data, p1_bbox_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)
    aligned_p2_df = interplate_and_align(eye_data, p2_bbox_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)

    aligned_p1_df["inbox"] = aligned_p1_df.apply(judge_inbox, axis=1)
    aligned_p2_df["inbox"] = aligned_p2_df.apply(judge_inbox, axis=1)
    aligned_p1_df["box_dist"] = aligned_p1_df.apply(compute_inbox_dist, axis=1)
    aligned_p2_df["box_dist"] = aligned_p2_df.apply(compute_inbox_dist, axis=1)
    
    res["Player1AttentionRatio"] = aligned_p1_df["inbox"].sum() / aligned_p1_df.shape[0]
    res["Player2AttentionRatio"] = aligned_p2_df["inbox"].sum() / aligned_p2_df.shape[0]
    res["Player1MinToCircle"] = aligned_p1_df["box_dist"].min()
    res["Player2MinToCircle"] = aligned_p2_df["box_dist"].min()

    return res

In [26]:
def extract_features(data, ball_data, player_box_data):
    data_df = pd.DataFrame(data).T
    data_df.ffill(inplace=True)
    data_df.bfill(inplace=True)

    ball_data_df = pd.DataFrame(ball_data)
    
    # match_rounds = label_round_hit(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id)

    match_rounds, rounds = threshold_find_match_round_dtw(data_df.copy(), ball_data_df.copy(), order=0, scale_raw_data=False, mode="fast")
    # match_rounds = find_match_round_hit(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id, time_range=7, dist=300)
    saccade_features = extract_features_round(match_rounds, data_df.copy(), ball_data_df.copy())

    attention_features = extract_features_whole(data_df.copy(), ball_data_df.copy(), player_box_data)
    
    return {
            "match_rounds" : match_rounds, 
            "rounds": rounds,
            "saccade_fea": saccade_features,
            "attention_fea": attention_features
        }

    # saccade_features = extract_saccade_features(match_rounds, data_df.loc[:, ["Screen.x", "Screen.y"]], video_id)
    # trajectory_features = extract_trajectory(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id, scale_to_percentage=True)

    # return {**saccade_features, **trajectory_features}

In [27]:
def add_features(data, ball_data, match_rounds):
    data_df = pd.DataFrame(data).T
    data_df.ffill(inplace=True)
    data_df.bfill(inplace=True)

    ball_data_df = pd.DataFrame(ball_data)
    
    saccade_features = add_saccade_features_round(match_rounds, data_df.copy(), ball_data_df.copy())
    
    return {
            "saccade_fea": saccade_features,
            "attention_fea": {},
        }

    # saccade_features = extract_saccade_features(match_rounds, data_df.loc[:, ["Screen.x", "Screen.y"]], video_id)
    # trajectory_features = extract_trajectory(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id, scale_to_percentage=True)

    # return {**saccade_features, **trajectory_features}

# Extract Features of all Participants

In [28]:
# all_people_fea = {}
# all_people_rounds = {}
# all_people_match_rounds = {}
# video_list= []

# for _person in all_data.keys():
#     if not _person == "24071512_AD": continue
#     _person_fea = {}
#     _person_match_rounds = {}
#     _person_rounds = {}

#     for _video in all_data[_person].keys():
#         # if not _video == "p7": continue
#         if not _video in video_list: video_list.append(_video)
        
#         res = extract_features(
#                 data=all_data[_person][_video], 
#                 ball_data=ball_data[_video.split("_")[0]],
#                 player_box_data=player_box_data[_video.split("_")[0]]
#             )
        
#         _person_fea[_video] = {}
#         _person_fea[_video]["All"] = {}
#         _person_fea[_video]["All"].update(res["attention_fea"])
#         _person_fea[_video]["All"].update({"MatchRoundRatio" : len(res["match_rounds"]) / len(res["rounds"])})
#         # _person_rounds[_video] = res["rounds"]
#         # if res["saccade_fea"]:
#         #     _person_fea[_video].update(res["saccade_fea"])
#         #     _person_match_rounds[_video] = res["match_rounds"]

#     all_people_fea[_person] = _person_fea
#     all_people_rounds[_person] = _person_rounds
#     all_people_match_rounds[_person] = _person_match_rounds

In [29]:
import json
import os

all_people_fea = {}
video_list= []

for _person in all_data.keys():
    if not _person == "24071512_AD": continue
    _person_fea = {}

    with open(os.path.join(FEA_DIR, f"{_person}/match_rounds.json"), "r") as f:
        people_match_round = json.load(f)
    for _video in all_data[_person].keys():
        # if not _video == "p7": continue
        if not _video in video_list: video_list.append(_video)
        res = add_features(
                data=all_data[_person][_video], 
                ball_data=ball_data[_video.split("_")[0]],
                match_rounds=people_match_round[_video]
            )
        _person_fea[_video] = {}
        if res["attention_fea"]:
            _person_fea[_video]["All"] = res["attention_fea"]
        if res["saccade_fea"]:
            _person_fea[_video].update(res["saccade_fea"])

    all_people_fea[_person] = _person_fea

In [30]:
def single_video_res(people_fea_res, video_id):
    res = {}
    for _p, person_fea in people_fea_res.items():
        try:
            for _r, _fea in person_fea[video_id].items():
                res[f"{_p}-{_r}"] = _fea
        except:
            continue
    return res


In [31]:
def single_person_res(people_fea_res, people_id):
    res = {}
    for _v, video_fea in people_fea_res[people_id].items():
        try:
            for _r, _fea in video_fea.items():
                res[f"{_v}-{_r}"] = _fea
        except:
            continue
    return res

In [32]:
def single_person_rounds(people_rounds_res, people_id):
    res = {}
    for _v, video_rounds in people_rounds_res[people_id].items():
        try:
            for _r, _fea in video_rounds.items():
                res[f"{_v}-{_r}"] = _fea
        except:
            continue
    return res

In [33]:
def single_person_match_rounds(people_match_rounds_res, people_id):
    res = {}
    for _v, video_rounds in people_match_rounds_res[people_id].items():
        try:
            for _r, _fea in video_rounds.items():
                res[f"{_v}-{_r}"] = _fea
        except:
            continue
    return res

In [34]:
person_fea = single_person_res(people_fea_res=all_people_fea, people_id="24071512_AD")
pd.DataFrame(person_fea).T

,SaccadeSpeed_Mean,SaccadeSpeed_Max,SaccadeSpeed_Min,SaccadeSpeed_Std,Amplitude,UnitAmplitude,SaccadeDelay,SaccadeDelayPercent,TrajDTW,TrajDTWPerBallMove,DirecAngle
w12_0_0_1-1.0,64.104802,762.506229,0.000000,137.453358,23.398238,1.852758,980.0,19.600000,0.661430,2.779614,15.163528
w12_0_0_1-2.0,50.129913,1523.821788,0.000000,245.900572,29.043948,1.044630,920.0,18.039216,1.056878,1.967410,8.172269
w12_0_0_1-3.0,36.157495,461.608837,0.000000,74.628901,24.663139,1.150270,980.0,10.315789,0.869435,1.534161,1.114373
w12_0_0_1-4.0,85.873243,1275.440924,0.000000,212.214308,21.528427,1.210197,620.0,13.777778,1.232535,2.334430,7.657090
w12_0_0_1-5.0,62.116215,444.931429,5.485357,102.161393,21.568835,0.997936,640.0,7.191011,0.324038,0.528516,9.811070
...,...,...,...,...,...,...,...,...,...,...,...
p15_A_A_3_1_2-18.0,64.883280,1064.923971,0.000000,215.060611,19.299083,0.575141,580.0,18.709677,0.678296,0.990757,5.021810
p15_A_A_3_1_2-19.0,65.030695,498.946566,0.197077,118.701437,21.140948,0.562368,580.0,19.333333,0.498096,0.624347,3.951447
p15_A_A_3_1_2-20.0,87.014189,507.129054,0.000000,150.130591,27.700102,0.723538,680.0,19.428571,0.692836,0.863334,8.997449
p15_A_A_3_1_2-21.0,85.801318,290.959354,7.616941,84.011246,10.555657,0.336605,580.0,19.333333,0.865735,1.362834,51.061187


In [35]:
video_id = video_list[0]
print(video_id)
video_res = single_video_res(people_fea_res=all_people_fea, video_id=video_id)
pd.DataFrame(video_res).T

w12_0_0_1


,SaccadeSpeed_Mean,SaccadeSpeed_Max,SaccadeSpeed_Min,SaccadeSpeed_Std,Amplitude,UnitAmplitude,SaccadeDelay,SaccadeDelayPercent,TrajDTW,TrajDTWPerBallMove,DirecAngle
24071512_AD-1.0,64.104802,762.506229,0.000000,137.453358,23.398238,1.852758,980.0,19.600000,0.661430,2.779614,15.163528
24071512_AD-2.0,50.129913,1523.821788,0.000000,245.900572,29.043948,1.044630,920.0,18.039216,1.056878,1.967410,8.172269
24071512_AD-3.0,36.157495,461.608837,0.000000,74.628901,24.663139,1.150270,980.0,10.315789,0.869435,1.534161,1.114373
24071512_AD-4.0,85.873243,1275.440924,0.000000,212.214308,21.528427,1.210197,620.0,13.777778,1.232535,2.334430,7.657090
24071512_AD-5.0,62.116215,444.931429,5.485357,102.161393,21.568835,0.997936,640.0,7.191011,0.324038,0.528516,9.811070
24071512_AD-6.0,43.886524,1165.767898,0.000000,174.307979,15.913705,0.863621,780.0,13.928571,1.384758,1.649755,2.933591
24071512_AD-7.0,71.991924,770.494920,0.000000,153.220113,22.542239,0.836918,540.0,6.835443,0.922653,1.595888,5.784888
24071512_AD-9.0,236.332743,1070.342434,0.513156,417.449346,19.550946,1.072144,920.0,10.454545,0.777243,1.236384,14.797497
24071512_AD-10.0,43.363032,583.147205,0.191941,106.312142,28.206077,2.200576,920.0,15.862069,0.607523,0.830595,28.334038
24071512_AD-13.0,30.685490,254.709136,0.473337,54.538783,18.293240,1.051062,720.0,11.076923,0.817107,2.553630,1.248250


# Save Results to Output

In [26]:
if not os.path.exists("output"):
    os.mkdir("output")

In [ ]:
import json

for file in file_list:
    if "Test" in file: continue
    people_id = file.split(".")[0]
    if not people_id in all_people_fea.keys(): continue
    person_fea = single_person_res(people_fea_res=all_people_fea, people_id=people_id)
    person_match_rounds = single_person_match_rounds(people_match_rounds_res=all_people_match_rounds, people_id=people_id)
    person_rounds = single_person_rounds(people_rounds_res=all_people_rounds, people_id=people_id)

    out_dir = f"output/{people_id}"
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    pd.DataFrame(person_fea).T.to_csv(f"{out_dir}/dtw_fast_features.csv")
    with open(f"{out_dir}/dtw_fast_rounds.json", "w") as f:
        json.dump(person_rounds, f)
    with open(f"{out_dir}/dtw_fast_match__rounds.json", "w") as f:
        json.dump(person_match_rounds, f)
